In [63]:
from nltk.corpus import semcor
import pandas as pd
import numpy as np
from semcor_bert_utils import *

In [3]:
tagged_sents = nltk.corpus.semcor.tagged_sents(tag = 'sem')

In [4]:
sel = SemCorSelector()

In [5]:
tag_freqs = {}
for s in tagged_sents:
    for tok in s:
        if type(tok) == nltk.tree.Tree:
            if tok.label() in tag_freqs.keys():
                tag_freqs[tok.label()] += 1
            else:
                tag_freqs[tok.label()] = 1
    

In [47]:
freqs = pd.DataFrame(pd.Series(tag_freqs)).reset_index()
freqs.columns = ['lemma', 'count']
sorted_freqs = freqs.sort_values('count', ascending = False)
words = []
def get_word(lem):
    if type(lem) == nltk.corpus.reader.wordnet.Lemma:
        return lem.name()
    else:
        return lem.split('.')[0]
sorted_freqs['word'] = sorted_freqs['lemma'].apply(get_word)

In [38]:
def get_pos(lem):
    lem = str(lem)
    try:
        return lem.split('.')[1]
    except:
        return 'No POS'

def get_sense_num(lem):
    lem = str(lem)
    try:
        return lem.split('.')[2]
    except:
        return "No marked sense"

In [43]:
tagged_sents[465][1].label()

Lemma('appointment.n.01.appointment')

In [48]:
decomp_tags = []
for s in tagged_sents:
    for tok in s:
        if type(tok) == nltk.tree.Tree:
            lem = tok.label()
            decomp_tags.append({'lemma': lem, 'word': get_word(lem), 
                                'pos': get_pos(lem), 'sense': get_sense_num(lem)})
            
semcor_tags = pd.DataFrame(decomp_tags)

In [49]:
semcor_tags

,lemma,word,pos,sense
0,Lemma('group.n.01.group'),group,n,01
1,Lemma('state.v.01.say'),say,v,01
2,Lemma('friday.n.01.Friday'),Friday,n,01
3,Lemma('probe.n.01.investigation'),investigation,n,01
4,Lemma('atlanta.n.01.Atlanta'),Atlanta,n,01
...,...,...,...,...
235074,Lemma('be.v.01.be'),be,v,01
235075,Lemma('let.v.01.let'),let,v,01
235076,Lemma('make.v.01.make'),make,v,01
235077,turn_into.v.00,turn_into,v,00


In [51]:
semcor_tags.to_csv('semcor_tags.csv')

In [149]:
freqs_for_sense = semcor_tags.groupby(["word", 'pos',
                                       'sense']).agg('count').rename({'lemma': 'freq'}, axis = 1).reset_index()
sparsity = freqs_for_sense.groupby(['word', 'pos']).agg([np.std, 'count'])

In [164]:
sparsity[('freq', 'count')] = sparsity['freq']['count'].apply(lambda x: x - 1)
sparsity = sparsity[sparsity['freq']['count'] > 1]
sparsity = sparsity.sort_values([('freq', 'std')], ascending = False).reset_index()

In [165]:
sparsity.head()

word pos         freq      
                     std count
0    be   v  2753.646765     8
1  have   v   341.328009     6
2    do   v   214.057214     2
3  make   v   176.710568     8
4   see   v   128.110418    13

In [166]:
sparsity.columns = ['word', 'pos', 'freq_sd', 'senses_for_pos']

In [170]:
sparsity.to_csv('data/semcor_sparsity.csv', index = False)